<a href="https://colab.research.google.com/github/AshwinKotgire/Play_with_LLMs/blob/main/Prompt_Engineering/LLM_UI_and_Cust_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U trl transformers accelerate
!pip install -q datasets bitsandbytes einops wandb
!pip install langchain
!pip install sentence_transformers
# !pip install faiss-cpu
!pip install faiss-gpu
!pip install pypdf
!pip install  tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install streamlit
!pip install pyngrok
!pip install hugchat

!pip install pypdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.3 MB/s eta 0:00:00
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=d0e9e8c64e18e1aaa754c6d3f24bdfeb9e77bc05d167f063a5942f8d3463cc95
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from Cust_chain import Cust_Chain_obj

ModuleNotFoundError: ignored

In [3]:
from huggingface_hub import login
login()

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline
import transformers
import torch

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import TokenTextSplitter

from IPython.display import HTML

from langchain.document_loaders import OnlinePDFLoader


from langchain.text_splitter import SentenceTransformersTokenTextSplitter

from Cust_chain import Cust_Chain_obj

In [2]:

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
sys_prompt="""<<SYS>>You are an honest assistant , who gives factually correct answers while refering to the context. If the context does not have an answer or if context is not available you try your best to answer the question on your own.<</SYS>>"""
chain=Cust_Chain_obj(model,tokenizer,None,sys_prompt)

In [4]:
url="https://arxiv.org/pdf/1706.03762.pdf"

chain.load_pdf_doc(url)
chain.populate_vector_store()


In [9]:
chain.populate_vector_store(chain.load_pdf_doc(url))

In [ ]:
chain.FAISS_obj.similarity_search('What are vectors?',k=2)

[Document(page_content='##ing, combined with fact that the output embeddings are offset by one position, ensures that the predictions for position ican depend only on the known outputs at positions less than i. 3. 2 attention an attention function can be described as mapping a query and a set of key - value pairs to an output, where the query, keys, values, and output are all vectors. the output is computed as a weighted sum 3', metadata={'source': '/tmp/tmpx1nmqjrx/tmp.pdf', 'page': 2}),
 Document(page_content='##duction model relying entirely on self - attention to compute representations of its input and output without using sequence - aligned rnns or convolution. in the following sections, we will describe the transformer, motivate self - attention and discuss its advantages over models such as [ 17, 18 ] and [ 9 ]. 3 model architecture most competitive neural sequence transduction models have an encoder - decoder structure [ 5, 2, 35 ]. here, the encoder maps an input sequence of 

In [ ]:
from IPython.display import Markdown

In [ ]:
Markdown(chain.run('What are transformers , explain in 100 words?',2,False)[1])

cuda


Transformers are a type of electrical device that transfers electrical energy from one circuit to another through electromagnetic induction. They are commonly used in power systems to increase or decrease the voltage of alternating current (AC) electrical energy. Transformers work by using two coils of wire, known as the primary and secondary coils, which are wrapped around a common magnetic core. When an alternating current flows through the primary coil, it generates a magnetic field that induces an electromotive force (EMF) in the secondary coil, causing an alternating current to flow in the secondary circuit. Transformers are used in a wide range of applications, including power generation and distribution, electronic devices, and electrical motors.

In [ ]:
class Chain(Cust_Chain_obj):
  def __init__(self,model,tokenizer,FAISS_obj,sys_prompt,url,embeddings = HuggingFaceEmbeddings(),
               text_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=50)):
    super.init(model,tokenizer,FAISS_obj,sys_prompt)
    self.url=url
    self.load_pdf(self.url)
    self.populate_vector_store()
    def load_new_doc_and_vector_store(self,url):
      self.url=url
      self.load_pdf
      self.popuolate_vector_store()
      return 'Successful'

In [ ]:
%%writefile A/main.py
print('Hello from A/main')
import numpy as np
import pandas as pd


Writing A/main.py


For loading pdf through streamlit components :https://stackoverflow.com/questions/76431655/langchain-pypdfloader

In [7]:
%%writefile chatbot.py
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline
import transformers
import torch

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
#from langchain.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader

from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import TokenTextSplitter

from IPython.display import HTML

from langchain.document_loaders import OnlinePDFLoader
from langchain.docstore.document import Document


from langchain.text_splitter import SentenceTransformersTokenTextSplitter

from Cust_chain import Cust_Chain_obj

import streamlit as st

@st.cache_resource
def load_model():

  model_name = "meta-llama/Llama-2-7b-chat-hf"

  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
  )

  model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
  )
  model.config.use_cache = False

  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
  tokenizer.pad_token = tokenizer.eos_token
  sys_prompt="""<<SYS>>You are an honest assistant , who gives factually correct answers if context is provided else you answer on your own.<</SYS>>"""
  chain=Cust_Chain_obj(model,tokenizer,None,sys_prompt)
  return chain

chain=load_model()
uploaded_file = st.file_uploader("Choose a file")
up_file=False
st.write(uploaded_file)


if (uploaded_file is not None):
  up_file=True
  pdf_reader = PdfReader(uploaded_file)
  docs=[]
  i=0
  for page in pdf_reader.pages:
    docs.append(Document(page_content=page.extract_text(), metadata={'page':i}))
    i+=1
  st.write(docs)
  chain.populate_vector_store(docs)


with st.sidebar:
  st.title('🤗💬 HugChat')
  st.success('Proceed to entering your prompt message!', icon='👉')
  st.markdown('📖 Learn how to build this app in this [blog](https://blog.streamlit.io/how-to-build-an-llm-powered-chatbot-with-streamlit/)!')

if "messages" not in st.session_state.keys():
  st.session_state.messages = [{"role": "assistant", "content": "How may I help you?"}]

# Display chat messages
for message in st.session_state.messages:
  with st.chat_message(message["role"]):
    st.write(message["content"])


# Function for generating LLM response
def generate_response(prompt_input, email, passwd):

  resp=chain.run(prompt_input,2,True)[1]
  return resp

# User-provided prompt
if prompt := st.chat_input():
  st.session_state.messages.append({"role": "user", "content": prompt})
  with st.chat_message("user"):
    st.write(prompt)

# Generate a new response if last message is not from assistant
if st.session_state.messages[-1]["role"] != "assistant":
  with st.chat_message("assistant"):
    with st.spinner("Thinking..."):
      response = generate_response(prompt, True, True)
      st.write("up_file true or not::",up_file)
      st.write(response)
  message = {"role": "assistant", "content": response}
  st.session_state.messages.append(message)




Overwriting chatbot.py


In [4]:
%%writefile chatbot.py



import streamlit as st

from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document


# Sidebar contents
with st.sidebar:
    st.title('🤗💬 LLM Chat App')
    st.markdown('''
    ## About
    This app is an LLM-powered chatbot built using:
    - [Streamlit](https://streamlit.io/)
    - [LangChain](https://python.langchain.com/)
    - [OpenAI](https://platform.openai.com/docs/models) LLM model
 ''')
    st.write('Made with ❤️ by [Prompt Engineer](https://youtube.com/@engineerprompt)')


def main():
    st.header("Chat with PDF 💬")


    # upload a PDF file
    pdf = st.file_uploader("Upload your PDF")

    st.write(pdf)
    if pdf is not None:
        pdf_reader = PdfReader(pdf)
        st.write(pdf_reader)
        docs=[]
        i=1
        text = ""
        for page in pdf_reader.pages:
          docs.append(Document(page_content=page.extract_text(), metadata={'page':i}))
          i+=1




if __name__ == '__main__':
    main()

Writing chatbot.py


In [4]:
from pyngrok import ngrok
ngrok.set_auth_token('2U6GDEjAMFtPezIMgNRPFzvPZGE_2EY17aWeK1e5jedmY1E9k')

In [5]:
!nohup streamlit run chatbot.py --server.port 200 &

url = ngrok.connect('200')
print(url)


nohup: appending output to 'nohup.out'


NgrokTunnel: "https://ef97-35-227-176-224.ngrok-free.app" -> "http://localhost:200"


In [16]:
%%writefile file_up.py
import streamlit as st
from PyPDF2 import PdfReader
from langchain.docstore.document import Document

uploaded_file = st.file_uploader("Upload your PDF")
if uploaded_file is not None:
    docs = []
    reader = PdfReader(uploaded_file)
    i = 1
    for page in reader.pages:
        docs.append(Document(page_content=page.extract_text(), metadata={'page':i}))
        i += 1
    st.write(docs)


Overwriting file_up.py


In [17]:
!nohup streamlit run file_up.py --server.port 300 &

url = ngrok.connect('300')
print(url)


nohup: appending output to 'nohup.out'
NgrokTunnel: "https://7197-35-227-176-224.ngrok-free.app" -> "http://localhost:300"
